# 4.10 Coding Etiquette & Excel Reporting Part 1

### This script contains the following points:
#### 1. Import resources
#### 2. Address PII data
#### 3. Create a regional segmentation of the data
#### 4. Create an exclusion flag for low-activity customers
#### 5. Create a profiling variable

Note: Part-1 delivers the end result of the dataframe "op_with_profiles" ready for visualization. Part-2 uses this result to create the correspondent visualizations.

### 1. Import resources

In [1]:
# Import libs
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Set a path variable for the import
path = r'C:\Users\zhoux\Documents\CF-Data Analyst Program\DA-Immersion\10-2023 Instacart Basket Analysis'

In [8]:
# Import data set
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_sub_customers.pkl'))

### 2.  Address PII data

In [6]:
# Display all columns in the dataframe to find anything related to PII
pd.set_option('display.max_columns', None)
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_prods_price,spending_flag,median_days_since_prior_order,regularity_flag,gender,state,age,n_dependents,fam_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busiest day,Least busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busiest day,Least busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423


The dataframe contains information about the customer's gender, state, age, number of dependents, family status and income. As we left out the name columns, these information is sensitive personal information but can't be used to directly identify a specific person. However, it is possible to put the pieces of information together, and it will be easy to figure out a customer's identity. 
We must to use a trustworthy encryption service and limit the data access strictly to people who are directly related to the data section of this project.

### 3. Create a regional segmentation of the data

In [7]:
# Create an empty list "result" to store the region value, 
# after categorizing the 'state' column in ords_prods_merge.
result = []
for value in ords_prods_merge['state']:
    if value == 'Maine' or value == 'New Hampshire' or value == 'Vermont' or \
        value == 'Massachusetts' or value == 'Rhode Island' or value == 'Connecticut' or \
        value == 'New York' or value == 'Pennsylvania' or value == 'New Jersey':
        result.append('Northeast')
    elif value == 'Wisconsin'or value == 'Michigan' or value =='Illinois' or \
          value == 'Indiana' or value == 'Ohio' or value == 'North Dakota' or \
          value == 'South Dakota' or value == 'Nebraska' or value == 'Kansas' or \
          value == 'Minnesota' or value == 'Iowa' or value == 'Missouri':
          result.append('Midwest')
    elif value == 'Delaware' or value == 'Maryland' or value == 'District of Columbia' or \
          value == 'Virginia' or value == 'West Virginia' or value == 'North Carolina' or \
          value == 'South Carolina' or value == 'Georgia' or value == 'Florida' or \
          value == 'Kentucky' or value == 'Tennessee' or value == 'Mississippi' or \
          value == 'Alabama' or value == 'Oklahoma' or value == 'Texas' or \
          value == 'Arkansas' or value == 'Louisiana':
          result.append('South')
    elif value == 'Idaho' or value == 'Montana' or value == 'Wyoming' or value == 'Nevada' or \
          value == 'Utah' or value == 'Colorado' or value == 'Arizona' or \
          value == 'New Mexico' or value == 'Alaska' or value == 'Washington' or \
          value == 'Oregon' or value == 'California' or value == 'Hawaii':
          result.append('West')

In [8]:
# Create a new column 'region' in ords_prods_merge and fill it with the values in 'result'
ords_prods_merge['region'] = result

In [9]:
# Look at value distributions to make sure there are no mistakes
ords_prods_merge['region'].value_counts(dropna = False)

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [10]:
crosstab_region_spending = pd.crosstab(ords_prods_merge['region'],
                           ords_prods_merge['spending_flag'], dropna = False)

In [11]:
crosstab_region_spending

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


Although the South region has the highest customer number, after further calculation in Excel, it seems that the percentage of high spenders is similar across all regions: </br>
Midwest - 2.05%, </br>
Northeast - 1.89%, </br>
South - 1.94%, </br>
West - 1.93%. </br>
There isn't a significant correlation between regions and spending habits.

### 4. Create an exclusion flag for low-activity customers

In [12]:
# Filter the customers with lesser than 5 orders and mark them as "exclude".
# To avoid nulls I also marked the rest as "Include".
ords_prods_merge.loc[ords_prods_merge['max_order'] < 5, 'low_act_exclusion_flag'] = 'Exclude'
ords_prods_merge.loc[ords_prods_merge['max_order'] >= 5, 'low_act_exclusion_flag'] = 'Include'

In [13]:
# Check whether the flag was successfully created
ords_prods_merge['low_act_exclusion_flag'].value_counts(dropna = False)

low_act_exclusion_flag
Include    30964564
Exclude     1440295
Name: count, dtype: int64

In [14]:
# Create a new dataframe that excludes the low activity customers
op_active_customers = ords_prods_merge[ords_prods_merge['low_act_exclusion_flag'] != 'Exclude']

In [15]:
# Check whether the new dataframe has the correct flag value
op_active_customers['low_act_exclusion_flag'].value_counts(dropna = False)

low_act_exclusion_flag
Include    30964564
Name: count, dtype: int64

In [16]:
# Export the new dataframe
op_active_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_active_customers.pkl'))

### 5. Create a profiling variable

From this point on, I will use op_active_customers, the dataframe that excludes low activity customers.

In [9]:
op_active_customers = op_active_customers.drop(['low_act_exclusion_flag'], axis=1)

In [11]:
op_active_customers.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,regularity_flag,gender,state,age,n_dependents,fam_status,income,region,age_group,income_group
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Non-frequent customer,Female,Alabama,31,3,married,40423,South,18-33,Low
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Non-frequent customer,Female,Alabama,31,3,married,40423,South,18-33,Low
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Non-frequent customer,Female,Alabama,31,3,married,40423,South,18-33,Low
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Non-frequent customer,Female,Alabama,31,3,married,40423,South,18-33,Low
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Non-frequent customer,Female,Alabama,31,3,married,40423,South,18-33,Low


The below steps create a profiling variable based on 
1. Age
2. Income
3. Certain goods in the “department_id” column
4. Number of dependent
5. Purchase of baby products and family status

#####  Create age groups:


In [12]:
# Look at the basic stats of Age
op_active_customers['age'].describe()

count    3.096456e+07
mean     4.946803e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: age, dtype: float64

We can see that the age ranges from 18-81. Based on the states, I divide the age in to 4 groups: 18-29. 30-39. 40-49, 50-59, 60-69, 70-81

In [26]:
op_active_customers.loc[(op_active_customers['age'] >= 18) & (op_active_customers['age'] <= 29), 'age_group'] = '18-29'

In [27]:
op_active_customers.loc[(op_active_customers['age'] >= 30) & (op_active_customers['age'] <= 39), 'age_group'] = '30-39'

In [28]:
op_active_customers.loc[(op_active_customers['age'] >= 40) & (op_active_customers['age'] <= 49), 'age_group'] = '40-49'    

In [29]:
op_active_customers.loc[(op_active_customers['age'] >= 50) & (op_active_customers['age'] <= 59), 'age_group'] = '50-59'    

In [30]:
op_active_customers.loc[(op_active_customers['age'] >= 60) & (op_active_customers['age'] <= 69), 'age_group'] = '60-69'    

In [31]:
op_active_customers.loc[(op_active_customers['age'] >= 70) & (op_active_customers['age'] <= 81), 'age_group'] = '70-81'    

In [32]:
# Check whether the grouping is successful
op_active_customers['age_group'].value_counts(dropna=False)

age_group
18-29    5817603
70-81    5812883
40-49    4883179
30-39    4847507
50-59    4841771
60-69    4761621
Name: count, dtype: int64

##### Create income groups:

In [29]:
# Look at the basic stats of Income
op_active_customers['income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

The stats shows that the income ranges from 25,903 - 593,901. 
Based on the stats, I divide the income into 4 groups:
1. 'Low' 25k-50k
2. 'Lower Middle' 50k-75k
3. 'Upper Middle' 75k-100k
4. 'High' 100k+

In [35]:
op_active_customers.loc[op_active_customers['income'] <= 50000, 'income_group'] = 'Low'

In [36]:
op_active_customers.loc[(op_active_customers['income'] > 50000) & (op_active_customers['income'] <= 75000), 'income_group'] = 'Lower Middle'

In [37]:
op_active_customers.loc[(op_active_customers['income'] > 75000) & (op_active_customers['income'] <= 100000), 'income_group'] = 'Upper Middle'

In [38]:
op_active_customers.loc[(op_active_customers['income'] > 100000), 'income_group'] = 'High'

In [39]:
# Check whether the grouping is successful
op_active_customers['income_group'].value_counts(dropna=False)

income_group
High            14207028
Upper Middle     6850802
Lower Middle     6544180
Low              3362554
Name: count, dtype: int64

##### Create groups of certain goods in the “department_id” column

In [19]:
# Import departments_wrangled.csv
df_departments = pd.read_csv(os.path.join(path,'02 Data', 'Prepared Data', 'departments_wrangled.csv'))

In [20]:
df_departments

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [21]:
df_departments.rename(columns = {'Unnamed: 0' : 'department_id'}, inplace = True)

In [22]:
df_departments.columns

Index(['department_id', 'department'], dtype='object')

In [23]:
op_active_customers['department_id'].value_counts(dropna=False)

department_id
4     9079273
16    5177182
19    2766406
7     2571901
1     2121731
13    1782705
3     1120828
15    1012074
20    1003834
9      822136
17     699857
12     674781
14     670850
11     424306
18     410392
6      255991
5      144627
8       93060
21      64768
2       34411
10      33451
Name: count, dtype: int64

In [24]:
# Merge the actual name of each department into the main dataframe based on department_id
op_active_customers = op_active_customers.merge(df_departments, on='department_id')

In [25]:
op_active_customers.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,gender,state,age,n_dependents,fam_status,income,region,age_group,income_group,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Female,Alabama,31,3,married,40423,South,18-33,Low,beverages
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Female,Alabama,31,3,married,40423,South,18-33,Low,beverages
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Female,Alabama,31,3,married,40423,South,18-33,Low,beverages
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Female,Alabama,31,3,married,40423,South,18-33,Low,beverages
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Female,Alabama,31,3,married,40423,South,18-33,Low,beverages


##### Create groups based on the number of dependent

In [11]:
op_active_customers['n_dependents'].value_counts(dropna=False)

n_dependents
3    7772516
0    7739681
2    7733261
1    7719106
Name: count, dtype: int64

In [12]:
# Find and group customers as "No dependents"
op_active_customers.loc[op_active_customers['n_dependents'] == 0, 'dependents_status'] = 'No Dependents' 

In [13]:
# Find and group customers as "1 or more dependents"
op_active_customers.loc[op_active_customers['n_dependents'] > 0, 'dependents_status'] = '1 or More Dependents' 

In [14]:
# Check the results of grouping
op_active_customers['dependents_status'].value_counts(dropna=False)

dependents_status
1 or More Dependents    23224883
No Dependents            7739681
Name: count, dtype: int64

In [15]:
op_active_customers.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,state,age,n_dependents,fam_status,income,region,age_group,income_group,department,dependents_status
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Alabama,31,3,married,40423,South,18-33,Low,beverages,1 or More Dependents
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Alabama,31,3,married,40423,South,18-33,Low,beverages,1 or More Dependents
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Alabama,31,3,married,40423,South,18-33,Low,beverages,1 or More Dependents
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Alabama,31,3,married,40423,South,18-33,Low,beverages,1 or More Dependents
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Alabama,31,3,married,40423,South,18-33,Low,beverages,1 or More Dependents


##### Create the customer's profile based on household status

Based on whether a customer purchases baby products and the family status of said customer, I group the customer into one of the six categories: </br>
1. Married household without baby(ies)
2. Single household without baby(ies)
3. Two-or-more person household without baby(ies)
4. Married household with baby(ies)
5. Single household with baby(ies)
6. Two-or-more person household with baby(ies)  

In [16]:
op_active_customers['fam_status'].value_counts(dropna=False)

fam_status
married                             21743711
single                               5094410
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: count, dtype: int64

If a customer has bought baby products, then we assume the customer's household has one or more babies. The following steps create a list of unique user_id that contains all customers with baby(ies).

In [37]:
# Create a subset containing only relevant columns
df_babies = op_active_customers[['user_id', 'department']]

In [38]:
df_babies.shape

(30964564, 2)

In [39]:
# Filter the subset by the babies department
df_babies = df_babies.loc[df_babies['department'] == 'babies']

In [41]:
# Create a new column to flag with/without baby(ies)
df_babies['baby_flag'] = 'with baby(ies)'
df_babies

,user_id,department,baby_flag
30486310,109,babies,with baby(ies)
30486311,290,babies,with baby(ies)
30486312,290,babies,with baby(ies)
30486313,420,babies,with baby(ies)
30486314,420,babies,with baby(ies)
...,...,...,...
30896697,194803,babies,with baby(ies)
30896698,194803,babies,with baby(ies)
30896699,21688,babies,with baby(ies)
30896700,21688,babies,with baby(ies)


In [42]:
# Delete the department column as it is not needed
df_babies = df_babies.drop(['department'], axis = 1)

In [43]:
# Delete all duplicate columns so that the subset only has unique rows
df_babies = df_babies.drop_duplicates()

In [44]:
df_babies

,user_id,baby_flag
30486310,109,with baby(ies)
30486311,290,with baby(ies)
30486313,420,with baby(ies)
30486319,778,with baby(ies)
30486323,1613,with baby(ies)
...,...,...
30896678,66087,with baby(ies)
30896679,11731,with baby(ies)
30896682,149691,with baby(ies)
30896690,194803,with baby(ies)


In [8]:
# Left join the main dataframe op_active_customers with the subset
op_active_customers = op_active_customers.join(df_babies.set_index('user_id'), on='user_id')

In [9]:
# Check whether the join was successful. The NaN values are customers who don't have baby(ies).
op_active_customers['baby_flag'].value_counts(dropna=False)

baby_flag
NaN               21154311
with baby(ies)     9810253
Name: count, dtype: int64

In [12]:
# Fill the baby_flag value for all customers who don't have baby(ies)
op_active_customers['baby_flag'].fillna('without baby(ies)', inplace=True)

In [13]:
# Check the flagging results
op_active_customers['baby_flag'].value_counts(dropna=False)

baby_flag
without baby(ies)    21154311
with baby(ies)        9810253
Name: count, dtype: int64

In [14]:
op_active_customers.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,n_dependents,fam_status,income,region,age_group,income_group,department,dependents_status,customer_profile,baby_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
2,473747,1,prior,3,3,12,21.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
4,431534,1,prior,5,4,15,28.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)


In [15]:
# Find and group customers as "Married household with baby(ies)"
op_active_customers.loc[(op_active_customers['baby_flag'] == 'with baby(ies)') & (op_active_customers['fam_status'] == 'married'), 'customer_profile'] = 'Married household with baby(ies)' 

In [16]:
# Find and group customers as "Single household with baby(ies)"
op_active_customers.loc[(op_active_customers['baby_flag'] == 'with baby(ies)') & ((op_active_customers['fam_status'] == 'single') | (op_active_customers['fam_status'] == 'divorced/widowed')) , 'customer_profile'] = 'Single household with baby(ies)' 

In [17]:
# Find and group customers as "Two-or-more person household with baby(ies)"
op_active_customers.loc[(op_active_customers['baby_flag'] == 'with baby(ies)') & (op_active_customers['fam_status'] == 'living with parents and siblings'), 'customer_profile'] = 'Two-or-more person household with baby(ies)' 

In [18]:
# Find and group customers as "Married household without baby(ies)"
op_active_customers.loc[(op_active_customers['baby_flag'] == 'without baby(ies)') & (op_active_customers['fam_status'] == 'married'), 'customer_profile'] = 'Married household without baby(ies)' 

In [19]:
# Find and group customers as "Single household without baby(ies)"
op_active_customers.loc[(op_active_customers['baby_flag'] == 'without baby(ies)') & ((op_active_customers['fam_status'] == 'single') | (op_active_customers['fam_status'] == 'divorced/widowed')) , 'customer_profile'] = 'Single household without baby(ies)' 

In [20]:
# Find and group customers as "Two-or-more person household without baby(ies)"
op_active_customers.loc[(op_active_customers['baby_flag'] == 'without baby(ies)') & (op_active_customers['fam_status'] == 'living with parents and siblings'), 'customer_profile'] = 'Two-or-more person household without baby(ies)' 

In [21]:
# Check the grouping results
op_active_customers['customer_profile'].value_counts(dropna=False)

customer_profile
Married household without baby(ies)               14854327
Married household with baby(ies)                   6889384
Single household without baby(ies)                 5283590
Single household with baby(ies)                    2456091
Two-or-more person household without baby(ies)     1016394
Two-or-more person household with baby(ies)         464778
Name: count, dtype: int64

In [22]:
# Check the new column "customer_profile"
op_active_customers.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,n_dependents,fam_status,income,region,age_group,income_group,department,dependents_status,customer_profile,baby_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
2,473747,1,prior,3,3,12,21.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)
4,431534,1,prior,5,4,15,28.0,196,1,1,...,3,married,40423,South,30-39,Low,beverages,1 or More Dependents,Married household without baby(ies),without baby(ies)


##### Export as .pkl

In [23]:
# Export the new dataframe
op_active_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'op_with_profiles.pkl'))